This code measures the GFP/RFP ratio, which is a measure of autophagy activation, with a lower ratio representing a higher autophagy rate as well as the number of inclusions in cells to compare chloroquine treatment with the untreated condition.

Import Libraries

In [12]:
import czifile
import numpy as np
from skimage.filters import gaussian, sobel, threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk
from scipy.ndimage import binary_fill_holes
from skimage.color import label2rgb
import matplotlib.pyplot as plt
import pandas as pd
import os

Define Sub Functions

In [17]:
def mask_near_border(mask, distance=5):
    """Checks if a binary mask comes within a specified distance of the image borders.

    Args:
        mask: A numpy array representing the binary mask (0s and 1s).
        distance: The distance from the border to check (default: 5 pixels).

    Returns:
        True if the mask comes within the specified distance of any border, False otherwise.
    """
    
    # Check if there are any True values in the specified range from the border
    if np.any(mask[:distance, :]) or np.any(mask[-distance:, :]): # Check if there are any Trues within distance from the top or bottom edge
        return True
    if np.any(mask[:, :distance]) or np.any(mask[:, -distance:]): # Check if there are any Trues within distance from the left or right edge
        return True
    
    # Otherwise, there are no Trues within 'distance' of the border
    return False

def process_file(file_path, basename):
    image = czifile.imread(file_path)
    image_squeezed = np.squeeze(image)
    first_frame = image_squeezed[0,:,:] #green channel
    second_frame = image_squeezed[1,:,:] #red channel
    red_channel = gaussian(first_frame, sigma=2)
    green_channel = gaussian(second_frame, sigma=2)
    initial_threshold = threshold_otsu(green_channel)
    cells_thresholded = green_channel > initial_threshold
    cells_edges = sobel(cells_thresholded)  # Using Sobel edge detection to highlight edges
    cells_edges = binary_dilation(cells_edges, disk(2))  # Increase border thickness
    cells_edges = binary_erosion(cells_edges, disk(1))  # Ensure borders are well-defined
    labeled_image = label(cells_edges)
    
    #initialize lists to store values
    red_wholecell_mfi = []
    green_wholecell_mfi = []
    num_inclusions = []
    cell_sizes = []
    
    i = 0 #counter to store regions
    # Iterate over regions
    for region in regionprops(labeled_image):
        # Check if region area is above the threshold
        cell_size = region.area
        if region.area < 1000:
            continue
        selem = disk(radius=3)
        # Create a mask for the current region
        mask = labeled_image == region.label
        if mask_near_border(mask):
            continue
        i = i + 1
        whole_cell = mask
        channel1_masked = green_channel * whole_cell
        channel2_masked = red_channel * whole_cell
        
        threshold = threshold_otsu(green_channel[whole_cell > 0])

        mask_inclusion = channel1_masked > threshold
        
        inclusion_edges = binary_dilation(mask_inclusion, disk(1))  # Increase border thickness
        labeled_inclusions = label(inclusion_edges) # Label Inclusions
        inclusion_counter = 0 # Initialize an inclusion counter
        for region in regionprops(labeled_inclusions):
            if region.area/cell_size > 0.3: # to account for segmentation issues
                continue
            inclusion_counter +=1
            
        # Calculate mean fluorescence intensity for each channel
        mean_intensity_channel1 = np.mean(channel1_masked[whole_cell])
        mean_intensity_channel2 = np.mean(channel2_masked[whole_cell])

        red_wholecell_mfi.append(mean_intensity_channel2)
        green_wholecell_mfi.append(mean_intensity_channel1)
        num_inclusions.append(inclusion_counter)
        cell_sizes.append(cell_size)

    # Create DataFrame to store values for the image
    df = pd.DataFrame({
        "Filename": [basename] * len(cell_sizes),
        "Cell": range(0, len(cell_sizes)),
        "red_wholecell_mfi": red_wholecell_mfi,
        "green_wholecell_mfi": green_wholecell_mfi,
        "num_inclusions": num_inclusions,
        'cell_size': cell_sizes,
        })
    return df


Define Main Function

In [18]:
all_data = []
discarded_images = []

folder_path = r"test_images"
for well_image in os.listdir(folder_path):
    if well_image.lower().endswith(".czi"):  # Filter for CZI files
        well_image_path = os.path.join(folder_path, well_image)
        well_image_base_name = os.path.basename(well_image)[:-4]
        cwd = os.getcwd()
        well_image_dir = os.path.join(cwd, well_image_base_name)
        df = process_file(well_image_path, well_image_base_name)
        if df is not None:
            all_data.append(df)
        else:
            discarded_images.append(well_image_base_name)

# Concatenate all DataFrames into one
combined_df = pd.concat(all_data, ignore_index=True)

# Save the combined DataFrame to a single CSV
combined_csv_path = os.path.join(cwd, "LC3.xlsx")
combined_df.to_excel(combined_csv_path, index=False)
